<a href="https://colab.research.google.com/github/ArtemDorofeev/Parse_elctro/blob/main/Era_char_parse.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# парсинг

In [ ]:
# Подключаем загрузку файла датасета с Гугл Диска

!gdown --id 1Hv0JJXuGrPLHv5z9TY8ZFjaRGmhHhb05

/usr/local/lib/python3.8/dist-packages/gdown/cli.py:127: FutureWarning: Option `--id` was deprecated in version 4.3.1 and will be removed in 5.0. You don't need to pass it anymore to use a file ID.
  warnings.warn(
Downloading...
From: https://drive.google.com/uc?id=1Hv0JJXuGrPLHv5z9TY8ZFjaRGmhHhb05
To: /content/prices.xlsx
100% 158k/158k [00:00<00:00, 70.8MB/s]


In [ ]:
import pandas as pd
import numpy as np
import requests
from bs4 import BeautifulSoup
import lxml
import json
import time
import csv


In [ ]:
# Записываем подгруженный файл excel в датафрейм, выводим информацию о нем

df = pd.read_excel('prices.xlsx')

In [ ]:
df = df[['Unnamed: 0.1', 'Unnamed: 3', 'Unnamed: 4', 'Unnamed: 5']]

In [ ]:
df = df.rename(columns={'Unnamed: 0.1': 'number', 'Unnamed: 3': 'art', 'Unnamed: 4': 'name', 'Unnamed: 5': 'link'})

In [ ]:
df

,number,art,name,link
0,1,Б0051088,Модуль светодиодный ЭРА Аксессуары SPB-6 с пул...,https://era-world.ru/modul-svetodiodnyy-era-ak...
1,2,Б0051089,Модуль светодиодный ЭРА Аксессуары SPB-6 с пул...,https://era-world.ru/modul-svetodiodnyy-era-ak...
2,3,Б0051090,Модуль светодиодный ЭРА Аксессуары SPB-6 с пул...,https://era-world.ru/modul-svetodiodnyy-era-ak...
3,4,Б0051091,Модуль светодиодный ЭРА Аксессуары SPB-6 с пул...,https://era-world.ru/modul-svetodiodnyy-era-ak...
4,5,Б0051103,Плафон ЭРА Подвесной к-т SPB-6 - подвесной ком...,https://era-world.ru/plafon-era-podvesnoy-k-t-...
...,...,...,...,...
2277,93,Б0052013,Светильник ЭРА НСП 01-60-003 подвесной Гранат ...,https://era-world.ru/svetil-nik-era-nsp-01-60-...
2278,94,Б0052017,Светильник ЭРА НСП 02-100-001 без решетки Желу...,https://era-world.ru/svetil-nik-era-nsp-02-100...
2279,95,Б0052019,Светильник ЭРА НСП 02-100-003 с решеткой Желуд...,https://era-world.ru/svetil-nik-era-nsp-02-100...
2280,96,Б0052018,Светильник ЭРА НСП 41-200-001 без решетки Желу...,https://era-world.ru/svetil-nik-era-nsp-41-200...


In [ ]:
# Формируем суп-объект

def get_soup(url):
  session = requests.session()
  session.headers = {
        "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/107.0.0.0 Safari/537.36",
        "Accept-language": "ru-RU,ru;q=0.9,en-US;q=0.8,en;q=0.7"
  }

  res = session.get(url)
  html = res.text
  soup = BeautifulSoup(html, "lxml")
  return soup


In [ ]:
d = df.sample(4)
d

,number,art,name,link,char,chardic
864,511,Б0049070,Лампочка светодиодная ЭРА STD LED MR16-6W-860-...,https://era-world.ru/lampochka-svetodiodnaya-e...,"[[Модель, Светоодача, Лм/Вт, Диаметр, Форма ко...","{'Модель': 'LED MR16-6W-860-GU10', 'Светоодача..."
605,252,Б0049641,Лампочка светодиодная ЭРА RED LINE LED B35-10W...,https://era-world.ru/lampochka-svetodiodnaya-e...,"[[Условия хранения, Светоотдача, Аналог лампы ...",{'Условия хранения': 'нет особых условий хране...
1073,34,Б0052773,Настольный светильник ЭРА NLED-497-12W-G свето...,https://era-world.ru/nastol-nyy-svetil-nik-era...,"[[Срок службы или срок годности, Класс защиты ...","{'Срок службы или срок годности': '-', 'Класс ..."
1746,89,Б0036497,DK LED 16 SL/WH Светильник ЭРА светодиодный кр...,https://era-world.ru/dk-led-16-sl-wh-svetil-ni...,"[[Световой поток, лм, Цвет подсветки, Условия ...","{'Световой поток, лм': '105', 'Цвет подсветки'..."


In [ ]:
char = []
table = soup.select('table.v-product-features__table')
nam = table[1].select('td.v-product-features__left')
val = table[1].select('td.v-product-features__right')
char.append(nam)
char.append(val)

In [ ]:
char

In [ ]:
def char_parse(row):
  try:    
    soup = get_soup(row)
    char = []
    table = soup.select('table.v-product-features__table')
    nam = table[1].select('td.v-product-features__left')
    val = table[1].select('td.v-product-features__right')
    char.append(nam)
    char.append(val)
    return char
  except:
    char = []
    return char

In [ ]:
def dic_make(row):
  dic = {}
  try:
    for i in range(len(row)):
      for j in range(len(row[i])):
        n = row[i][j].get_text()
        row[i][j] = n
    for i in range(len(row[0])):
      y = {row[0][i]: row[1][i]}
      dic.update(y)
  except:
    dic
  return dic


In [ ]:
df['char'] = df['link'].apply(char_parse)

In [ ]:
df

,number,art,name,link,char
0,1,Б0051088,Модуль светодиодный ЭРА Аксессуары SPB-6 с пул...,https://era-world.ru/modul-svetodiodnyy-era-ak...,"[[[<span class=""v-product-features__name"">Гара..."
1,2,Б0051089,Модуль светодиодный ЭРА Аксессуары SPB-6 с пул...,https://era-world.ru/modul-svetodiodnyy-era-ak...,"[[[<span class=""v-product-features__name"">Габа..."
2,3,Б0051090,Модуль светодиодный ЭРА Аксессуары SPB-6 с пул...,https://era-world.ru/modul-svetodiodnyy-era-ak...,"[[[<span class=""v-product-features__name"">Габа..."
3,4,Б0051091,Модуль светодиодный ЭРА Аксессуары SPB-6 с пул...,https://era-world.ru/modul-svetodiodnyy-era-ak...,"[[[<span class=""v-product-features__name"">Цвет..."
4,5,Б0051103,Плафон ЭРА Подвесной к-т SPB-6 - подвесной ком...,https://era-world.ru/plafon-era-podvesnoy-k-t-...,"[[[<span class=""v-product-features__name"">Реко..."
...,...,...,...,...,...
2277,93,Б0052013,Светильник ЭРА НСП 01-60-003 подвесной Гранат ...,https://era-world.ru/svetil-nik-era-nsp-01-60-...,"[[[<span class=""v-product-features__name"">Габа..."
2278,94,Б0052017,Светильник ЭРА НСП 02-100-001 без решетки Желу...,https://era-world.ru/svetil-nik-era-nsp-02-100...,"[[[<span class=""v-product-features__name"">Мате..."
2279,95,Б0052019,Светильник ЭРА НСП 02-100-003 с решеткой Желуд...,https://era-world.ru/svetil-nik-era-nsp-02-100...,"[[[<span class=""v-product-features__name"">Мате..."
2280,96,Б0052018,Светильник ЭРА НСП 41-200-001 без решетки Желу...,https://era-world.ru/svetil-nik-era-nsp-41-200...,"[[[<span class=""v-product-features__name"">Исто..."


In [ ]:
df['chardic'] = df['char'].apply(dic_make)

In [ ]:
# Записываем объедененные csv в итоговый Excel

writer = pd.ExcelWriter('char_parse4.xlsx') # write dataframe to excel 
dataset.to_excel(writer) # save the excel 
writer.save()

# обработка

In [ ]:
# Подключаем загрузку файла датасета с Гугл Диска

!gdown --id 1JKD_zYnhAa4apnrJPGdSnZmcq7iAHPTB

/usr/local/lib/python3.8/dist-packages/gdown/cli.py:127: FutureWarning: Option `--id` was deprecated in version 4.3.1 and will be removed in 5.0. You don't need to pass it anymore to use a file ID.
  warnings.warn(
Downloading...
From: https://drive.google.com/uc?id=1JKD_zYnhAa4apnrJPGdSnZmcq7iAHPTB
To: /content/char_parse.xlsx
100% 634k/634k [00:00<00:00, 97.4MB/s]


In [ ]:
import pandas as pd
import numpy as np
import requests
from bs4 import BeautifulSoup
import lxml
import json
import time
import csv


In [ ]:
# Записываем подгруженный файл excel в датафрейм, выводим информацию о нем

df = pd.read_excel('char_parse.xlsx')

In [ ]:
df.columns

Index(['Unnamed: 0', 'number', 'art', 'name', 'link', 'char', 'chardic'], dtype='object')

In [ ]:
dfr = df[['number', 'art', 'name', 'link', 'chardic']]

In [ ]:
dfr['chardic'] = dfr['chardic'].apply(eval)

<ipython-input-20-56e5ea42eb74>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dfr['chardic'] = dfr['chardic'].apply(eval)


In [ ]:
lst = ['мощность', 'световой поток', 'цветовая температура', 'цокол', 'колбы']

In [ ]:
def char_podbor(row):
  chr = {}
  for i in lst:
    for j in row:
      if i in j.lower():
        y = {j: row[j]}
        chr.update(y)
  return chr 

In [ ]:
dfr['chr_lst'] = dfr['chardic'].apply(char_podbor)

<ipython-input-23-5978be2a6889>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dfr['chr_lst'] = dfr['chardic'].apply(char_podbor)


In [ ]:
dfr

In [ ]:
dataset = dfr[['number', 'art', 'name', 'link', 'chr_lst']]

In [ ]:
dataset

In [ ]:
def char_pow(row):
  sep = {}
  for i in row:
    if 'мощность' in i.lower():
      x = {i: row[i]}
      sep.update(x)
  return sep

def char_sp(row):
  sep = {}
  for i in row:
    if 'световой поток' in i.lower():
      x = {i: row[i]}
      sep.update(x)
  return sep

def char_ct(row):
  sep = {}
  for i in row:
    if 'цветовая температура' in i.lower():
      x = {i: row[i]}
      sep.update(x)
  return sep

def char_zok(row):
  sep = {}
  for i in row:
    if 'цокол' in i.lower():
      x = {i: row[i]}
      sep.update(x)
  return sep

def char_lam(row):
  sep = {}
  for i in row:
    if 'колбы' in i.lower():
      x = {i: row[i]}
      sep.update(x)
  return sep

In [ ]:
dataset['power'] = dataset['chr_lst'].apply(char_pow)
dataset['svet_potok'] = dataset['chr_lst'].apply(char_sp)
dataset['cvet_temper'] = dataset['chr_lst'].apply(char_ct)
dataset['cokol'] = dataset['chr_lst'].apply(char_zok)
dataset['form_lamp'] = dataset['chr_lst'].apply(char_lam)

In [ ]:
dataset

In [ ]:
# Записываем объедененные csv в итоговый Excel

writer = pd.ExcelWriter('char_parse5.xlsx') # write dataframe to excel
dataset.to_excel(writer) # save the excel
writer.save()